In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

scanpy==1.8.1 anndata==0.7.6 umap==0.5.1 numpy==1.20.3 scipy==1.7.1 pandas==1.3.2 scikit-learn==0.24.2 statsmodels==0.12.2 python-igraph==0.9.6 pynndescent==0.5.4


In [2]:
rna = sc.read_h5ad("/home/bram/jointomicscomp/data/pbmc_multimodal_RNA.h5ad")

/home/bram/Documents/anaconda3/envs/vae/lib/python3.9/site-packages/anndata/compat/__init__.py:180: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  warn(


In [3]:
rna
# sc.pp.highly_variable_genes(rna, 5000)

AnnData object with n_obs × n_vars = 161764 × 20729
    obs: 'nCount_ADT', 'nFeature_ADT', 'nCount_RNA', 'nFeature_RNA', 'orig.ident', 'lane', 'donor', 'time', 'celltype.l1', 'celltype.l2', 'celltype.l3', 'Phase'
    var: 'features'
    uns: 'neighbors'
    obsm: 'X_apca', 'X_aumap', 'X_pca', 'X_spca', 'X_umap', 'X_wnn.umap'
    varm: 'PCs', 'SPCA'
    obsp: 'distances'

In [5]:
rna.obs

,nCount_ADT,nFeature_ADT,nCount_RNA,nFeature_RNA,orig.ident,lane,donor,time,celltype.l1,celltype.l2,celltype.l3,Phase
L1_AAACCCAAGAAACTCA,7535.0,217,10823.0,2915,P2_7,L1,P2,7,Mono,CD14 Mono,CD14 Mono,G1
L1_AAACCCAAGACATACA,6013.0,209,5864.0,1617,P1_7,L1,P1,7,CD4 T,CD4 TCM,CD4 TCM_1,G1
L1_AAACCCACAACTGGTT,6620.0,213,5067.0,1381,P4_3,L1,P4,3,CD8 T,CD8 Naive,CD8 Naive,S
L1_AAACCCACACGTACTA,3567.0,202,4786.0,1890,P3_7,L1,P3,7,NK,NK,NK_2,G1
L1_AAACCCACAGCATACT,6402.0,215,6505.0,1621,P4_7,L1,P4,7,CD8 T,CD8 Naive,CD8 Naive,G1
...,...,...,...,...,...,...,...,...,...,...,...,...
E2L8_TTTGTTGGTCGTGATT,4170.0,197,9346.0,2201,P5_7,E2L8,P5,7,CD8 T,CD8 Naive,CD8 Naive,S
E2L8_TTTGTTGGTGTGCCTG,6927.0,209,9318.0,2938,P5_3,E2L8,P5,3,Mono,CD14 Mono,CD14 Mono,G1
E2L8_TTTGTTGGTTAGTTCG,4222.0,206,11619.0,3224,P8_0,E2L8,P8,0,B,B intermediate,B intermediate kappa,S
E2L8_TTTGTTGGTTGGCTAT,6063.0,217,15436.0,3999,P5_3,E2L8,P5,3,Mono,CD16 Mono,CD16 Mono,G1


In [11]:
print(rna.obs['celltype.l1'].value_counts())
print("\n")
print(rna.obs['celltype.l2'].value_counts())
print("\n")

print(rna.obs['celltype.l3'].value_counts())



Mono       49010
CD4 T      41001
CD8 T      25469
NK         18664
B          13800
other T     6789
DC          3589
other       3442
Name: celltype.l1, dtype: int64


CD14 Mono            42690
CD4 Naive            17479
NK                   17173
CD4 TCM              14889
CD8 TEM              11727
CD8 Naive            10768
B naive               7718
CD16 Mono             6320
CD4 TEM               4282
gdT                   3649
B memory              3285
CD8 TCM               2883
MAIT                  2784
Treg                  2507
cDC2                  2501
B intermediate        2431
Platelet              2293
CD4 CTL               1736
NK_CD56bright          943
pDC                    861
Doublet                605
NK Proliferating       548
Plasmablast            366
dnT                    356
HSPC                   329
cDC1                   151
ILC                    132
CD4 Proliferating      108
CD8 Proliferating       91
Eryth                   83
ASDC                

In [6]:
# Create predefined splits for testing purposes
from sklearn.model_selection import StratifiedShuffleSplit

split1 = StratifiedShuffleSplit(n_splits=1, test_size=0.1)

trainValidInd, testInd = split1.split(X, y)

Xtest = X[testInd]

ytest = y[testInd]

XtrainValid = X[trainValidInd]

ytrainValid = y[trainValidInd]

split2 = StratifiedShuffleSplit(n_splits=1, test_size=1/9)

trainInd, validInd = split1.split(XtrainValid, ytrainValid)

Xtrain = XtrainValid[trainInd]

ytrain = ytrainValid[trainInd]

Xvalid = XtrainValid[validInd]

yvalid = ytrainValid[validInd]